In [334]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from urllib.parse import quote
import requests
import json
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from google.colab import drive
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import ast
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import warnings
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error, r2_score,root_mean_squared_error,root_mean_squared_log_error
from sklearn.metrics import accuracy_score
import xgboost as xgb
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle


# Para usar no Colab

In [217]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Obtendo Dados

In [218]:
df = pd.read_csv("drive/My Drive/Indicium/Trabalho Indicium/Arquivos csv/desafio_indicium_imdb.csv")
df.drop(columns="Unnamed: 0",inplace=True)
df.fillna(value=0,inplace=True)

In [219]:
df.head(5)

,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross
0,The Godfather,1972,A,175 min,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,100.0,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1620367,"134,966,411"
1,The Dark Knight,2008,UA,152 min,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,84.0,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,2303232,"534,858,444"
2,The Godfather: Part II,1974,A,202 min,"Crime, Drama",9.0,The early life and career of Vito Corleone in ...,90.0,Francis Ford Coppola,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton,1129952,"57,300,000"
3,12 Angry Men,1957,U,96 min,"Crime, Drama",9.0,A jury holdout attempts to prevent a miscarria...,96.0,Sidney Lumet,Henry Fonda,Lee J. Cobb,Martin Balsam,John Fiedler,689845,"4,360,000"
4,The Lord of the Rings: The Return of the King,2003,U,201 min,"Action, Adventure, Drama",8.9,Gandalf and Aragorn lead the World of Men agai...,94.0,Peter Jackson,Elijah Wood,Viggo Mortensen,Ian McKellen,Orlando Bloom,1642758,"377,845,905"


## Pré Processamento

In [220]:
df.dtypes

,0
Series_Title,object
Released_Year,object
Certificate,object
Runtime,object
Genre,object
IMDB_Rating,float64
Overview,object
Meta_score,float64
Director,object
Star1,object


In [221]:
## check if runtime is all in minutes, e o espaço
hora = -1
for i in range(0,len(df)):
    value = df.iloc[i,3][-4:]
    if value != " min":
        hora = 1
print("Todos os valores estão na unidade minuto" if hora == -1 else "Tem algum valor que não esta na unidade minuto")

Todos os valores estão na unidade minuto


In [222]:
df.head(1)

,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross
0,The Godfather,1972,A,175 min,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,100.0,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1620367,"134,966,411"


In [223]:
## tirando o minuto da coluna

for i in range(0,len(df)):
    value = df.iloc[i,3][-4:]
    if value == " min":
        df.iloc[i,3] = df.iloc[i,3][:-4]

df.rename(columns={"Runtime":"Runtime (min)"},inplace=True)


In [224]:
df["Released_Year"].unique()

array(['1972', '2008', '1974', '1957', '2003', '1994', '1993', '2010',
       '1999', '2001', '1966', '2002', '1990', '1980', '1975', '2020',
       '2019', '2014', '1998', '1997', '1995', '1991', '1977', '1962',
       '1954', '1946', '2011', '2006', '2000', '1988', '1985', '1968',
       '1960', '1942', '1936', '1931', '2018', '2017', '2016', '2012',
       '2009', '2007', '1984', '1981', '1979', '1971', '1963', '1964',
       '1950', '1940', '2013', '2005', '2004', '1992', '1987', '1986',
       '1983', '1976', '1973', '1965', '1959', '1958', '1952', '1948',
       '1944', '1941', '1927', '1921', '2015', '1996', '1989', '1978',
       '1961', '1955', '1953', '1925', '1924', '1982', '1967', '1951',
       '1949', '1939', '1937', '1934', '1928', '1926', '1920', '1970',
       '1969', '1956', '1947', '1945', '1930', '1938', '1935', '1933',
       '1932', '1922', '1943', 'PG'], dtype=object)

In [225]:
df[df["Released_Year"]=="PG"].index[0]

np.int64(965)

In [226]:
# verificando porque deu erro para transformar a coluna released year em float
# temos um valor PG na coluna, o mesmo não faz sentido
# filtrando (df[df["Released_Year"]=="PG"]) podemos ver que se trata do filme
# Apollo 13, sendo assim pesquisei e no dataset esta errado
# alterei entao para a data correta 1995
df.iloc[df[df["Released_Year"]=="PG"].index[0],df.columns.to_list().index("Released_Year")] = "1995"
# podemos ver que esta corretamente corrigido
df[df["Series_Title"]=="Apollo 13"]

,Series_Title,Released_Year,Certificate,Runtime (min),Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross
965,Apollo 13,1995,U,140,"Adventure, Drama, History",7.6,NASA must devise a strategy to return Apollo 1...,77.0,Ron Howard,Tom Hanks,Bill Paxton,Kevin Bacon,Gary Sinise,269197,"173,837,933"


In [227]:
df['Gross']

,Gross
0,"134,966,411"
1,"534,858,444"
2,"57,300,000"
3,"4,360,000"
4,"377,845,905"
...,...
994,0
995,0
996,"30,500,000"
997,0


In [228]:
type(df.iloc[10,df.columns.to_list().index("Gross")])

str

In [229]:
# para colocar a coluna Gross como float
# Devemos retirar a virgula, pois ele esta neste formato 134,966,411

for i in range(0,len(df)):
    #print(type(df.iloc[i,df.columns.to_list().index("Gross")]))
    if type(df.iloc[i,df.columns.to_list().index("Gross")]) == str:
        df.iloc[i,df.columns.to_list().index("Gross")] = df.iloc[i,df.columns.to_list().index("Gross")].replace(",","")
        #print(df.iloc[i,df.columns.to_list().index("Gross")])

In [230]:
df['Gross'].unique()

array(['134966411', '534858444', '57300000', '4360000', '377845905',
       '107928762', '96898818', '292576195', '37030102', '315544750',
       '330252182', '6100000', '342551365', '171479930', '46836394',
       '290475067', '112000000', 0, '53367844', '188020017', '7563397',
       '10055859', '216540909', '136801374', '57598247', '100125643',
       '130742922', '322740140', '269061', '335451311', '13092000',
       '13182281', '53089891', '132384315', '32572577', '187705427',
       '6719864', '23341568', '19501238', '422783777', '204843350',
       '11990401', '210609762', '5321508', '32000000', '1024560',
       '163245', '19181', '1661096', '5017246', '12391761', '190241310',
       '858373000', '678815482', '209726015', '162805434', '448139099',
       '6532908', '1223869', '223808164', '11286112', '707481',
       '25544867', '2375308', '248159971', '44017374', '83471511',
       '78900000', '275902', '8175000', '36764313', '288475', '159227644',
       '1373943', '687185', 

In [231]:
# colocar o tipo como float
df = df.astype({"Runtime (min)":"float64","Released_Year":"float64","Gross":"float64"})

In [232]:
df.dtypes

,0
Series_Title,object
Released_Year,float64
Certificate,object
Runtime (min),float64
Genre,object
IMDB_Rating,float64
Overview,object
Meta_score,float64
Director,object
Star1,object


In [233]:
a = df.iloc[1,df.columns.to_list().index("Genre")]


primeira = a[:a.index(",")]
segunda = a[len(primeira) + 2:]
segunda

'Crime, Drama'

In [234]:
df.columns

Index(['Series_Title', 'Released_Year', 'Certificate', 'Runtime (min)',
       'Genre', 'IMDB_Rating', 'Overview', 'Meta_score', 'Director', 'Star1',
       'Star2', 'Star3', 'Star4', 'No_of_Votes', 'Gross'],
      dtype='object')

In [235]:
# Quebrar os gêneros em colunas separadas automaticamente
genres_split = df["Genre"].str.split(", ", expand=True)

# Renomear as colunas
genres_split.columns = [f"Genre {i+1}" for i in range(genres_split.shape[1])]

# Concatenar com o dataframe original
df = pd.concat([df, genres_split], axis=1)

#df.drop(columns="Genre",inplace=True)

df = df[['Series_Title', 'Released_Year', 'Certificate', 'Runtime (min)','Genre',
        'Genre 1', 'Genre 2','Genre 3','IMDB_Rating',
        'Overview', 'Meta_score', 'Director', 'Star1',
       'Star2', 'Star3', 'Star4', 'No_of_Votes', 'Gross']]


In [236]:
df.head(5)

,Series_Title,Released_Year,Certificate,Runtime (min),Genre,Genre 1,Genre 2,Genre 3,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross
0,The Godfather,1972.0,A,175.0,"Crime, Drama",Crime,Drama,None,9.2,An organized crime dynasty's aging patriarch t...,100.0,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1620367,134966411.0
1,The Dark Knight,2008.0,UA,152.0,"Action, Crime, Drama",Action,Crime,Drama,9.0,When the menace known as the Joker wreaks havo...,84.0,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,2303232,534858444.0
2,The Godfather: Part II,1974.0,A,202.0,"Crime, Drama",Crime,Drama,None,9.0,The early life and career of Vito Corleone in ...,90.0,Francis Ford Coppola,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton,1129952,57300000.0
3,12 Angry Men,1957.0,U,96.0,"Crime, Drama",Crime,Drama,None,9.0,A jury holdout attempts to prevent a miscarria...,96.0,Sidney Lumet,Henry Fonda,Lee J. Cobb,Martin Balsam,John Fiedler,689845,4360000.0
4,The Lord of the Rings: The Return of the King,2003.0,U,201.0,"Action, Adventure, Drama",Action,Adventure,Drama,8.9,Gandalf and Aragorn lead the World of Men agai...,94.0,Peter Jackson,Elijah Wood,Viggo Mortensen,Ian McKellen,Orlando Bloom,1642758,377845905.0


In [237]:
### agora devo agrupar para entender o que pode gerar mais dinheiro, agrupar por genero, ator classificação indicativa,diretor
## ficar atento com os atores, pois é interessante juntar o ATOR, não importa se ele esta como star 1 ou star n
## pensar nessa logica

In [238]:
df["Runtime (min)"][df["Genre 1"] == "Action"].sum()/len(df[["Genre 1","Runtime (min)"]][df["Genre 1"] == "Action"])

np.float64(129.04651162790697)

In [239]:
df[df["Genre 1"] == "Fantasy"]

,Series_Title,Released_Year,Certificate,Runtime (min),Genre,Genre 1,Genre 2,Genre 3,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross
320,Das Cabinet des Dr. Caligari,1920.0,0,76.0,"Fantasy, Horror, Mystery",Fantasy,Horror,Mystery,8.1,"Hypnotist Dr. Caligari uses a somnambulist, Ce...",0.0,Robert Wiene,Werner Krauss,Conrad Veidt,Friedrich Feher,Lil Dagover,57428,0.0
567,Nosferatu,1922.0,0,94.0,"Fantasy, Horror",Fantasy,Horror,None,7.9,Vampire Count Orlok expresses interest in a ne...,0.0,F.W. Murnau,Max Schreck,Alexander Granach,Gustav von Wangenheim,Greta Schröder,88794,0.0


In [240]:
### pegando a tabela de detalhes feitas no dada eng

tmdb_id = pd.read_csv("drive/My Drive/Indicium/Trabalho Indicium/Arquivos csv/id_tmdb_final.csv")
movie_details = pd.read_csv("drive/My Drive/Indicium/Trabalho Indicium/Arquivos csv/movies_tmdb_detalhes.csv")
#transforma o belongs to colection em binario
movie_details["belongs_to_collection_bin"] = movie_details["belongs_to_collection_name"].notna().astype("int32")



# Análise em cima de Gêneros x Lucro

In [241]:
## algum dos titulos esta duplicado

duplicados_df = df[df["Series_Title"].duplicated(keep=False)]
print(duplicados_df.sort_values("Series_Title"))

# no caso o fime Drishyam, mesmo nome mas filmes diferentes

    Series_Title  Released_Year Certificate  Runtime (min)  \
86      Drishyam         2013.0           U          160.0   
135     Drishyam         2015.0          UA          163.0   

                      Genre Genre 1 Genre 2   Genre 3  IMDB_Rating  \
86   Crime, Drama, Thriller   Crime   Drama  Thriller          8.3   
135   Crime, Drama, Mystery   Crime   Drama   Mystery          8.2   

                                              Overview  Meta_score  \
86   A man goes to extreme lengths to save his fami...         0.0   
135  Desperate measures are taken by a man who trie...         0.0   

            Director       Star1         Star2         Star3         Star4  \
86     Jeethu Joseph    Mohanlal         Meena  Asha Sharath        Ansiba   
135  Nishikant Kamat  Ajay Devgn  Shriya Saran          Tabu  Rajat Kapoor   

     No_of_Votes     Gross  
86         30722       0.0  
135        70367  739478.0  


In [242]:
### para a analise de genero x lucro iremos usar os dados obtidos da api
# temos que transformar o data no tmdb_id para fazer o merge
# primeiro tem que ajeitar aquela data que ta como PG

tmdb_id.iloc[tmdb_id[tmdb_id["data_no_dataset"]=="PG"].index[0],tmdb_id.columns.to_list().index("data_no_dataset")] = "1995"
tmdb_id = tmdb_id.astype({"data_no_dataset":"float64"})
intermediaria = df.merge(tmdb_id[["id","nome_no_dataset","data_no_dataset"]],how="inner",left_on=["Series_Title","Released_Year"],right_on=["nome_no_dataset","data_no_dataset"])
tabela_geral = intermediaria.merge(movie_details,how="inner",left_on="id",right_on="id")

# Considerar budgets menores que 100k como 0
tabela_geral.loc[tabela_geral['budget'] < 100000, 'budget'] = 0
## analisando muitos budgets estao errados na TMDb, entao irei apenas considerar os que sao maiores que
## 100k, achei um minimo razoavel
tabela_geral["profit overall"] = tabela_geral["Gross"] - tabela_geral["budget"]
tabela_geral.fillna(0,inplace=True)


In [243]:
tabela_geral.Released_Year

,Released_Year
0,1972.0
1,2008.0
2,1974.0
3,1957.0
4,2003.0
...,...
993,1961.0
994,1956.0
995,1953.0
996,1944.0


para 157 filmes o budget esta como 0, entao nao devem ter consiguido obter o valor a tmdb

In [244]:

tabela_geral["Count"] = 1
df_genre_1 = tabela_geral.groupby(['Genre 1'])[['Gross',"budget","revenue",'profit overall','belongs_to_collection_bin','Count']].sum().reset_index()
df_genre_2 = tabela_geral.groupby(['Genre 2'])[['Gross',"budget","revenue",'profit overall','belongs_to_collection_bin','Count']].sum().reset_index()
df_genre_3 = tabela_geral.groupby(['Genre 3'])[['Gross',"budget","revenue",'profit overall','belongs_to_collection_bin','Count']].sum().reset_index()
# para ter o numero de filmes e a media de bilheteria
df_genre_1["Gross_mean"] = df_genre_1["Gross"]/df_genre_1["Count"]
df_genre_2["Gross_mean"] = df_genre_2["Gross"]/df_genre_2["Count"]
df_genre_3["Gross_mean"] = df_genre_3["Gross"]/df_genre_3["Count"]

df_genre_1["budget_mean"] = df_genre_1["budget"]/df_genre_1["Count"]
df_genre_2["budget_mean"] = df_genre_2["budget"]/df_genre_2["Count"]
df_genre_3["budget_mean"] = df_genre_3["budget"]/df_genre_3["Count"]


# lucro percentual correto por gênero
df_genre_1["profit (%)"] = 100 * df_genre_1["profit overall"] / df_genre_1["budget"]
df_genre_2["profit (%)"] = 100 * df_genre_2["profit overall"] / df_genre_2["budget"]
df_genre_3["profit (%)"] = 100 * df_genre_3["profit overall"] / df_genre_3["budget"]

# evitar infinitos caso budget = 0
df_genre_1["profit (%)"] = df_genre_1["profit (%)"].replace([float("inf"), -float("inf")], 0)
df_genre_2["profit (%)"] = df_genre_2["profit (%)"].replace([float("inf"), -float("inf")], 0)
df_genre_3["profit (%)"] = df_genre_3["profit (%)"].replace([float("inf"), -float("inf")], 0)

# média de lucro absoluto
df_genre_1["profit mean"] = df_genre_1["Gross_mean"] - df_genre_1["budget_mean"]
df_genre_2["profit mean"] = df_genre_2["Gross_mean"] - df_genre_2["budget_mean"]
df_genre_3["profit mean"] = df_genre_3["Gross_mean"] - df_genre_3["budget_mean"]

df_genre_1["belongs_to_collection_bin"] = df_genre_1["belongs_to_collection_bin"]/df_genre_1["Count"]
df_genre_2["belongs_to_collection_bin"] = df_genre_2["belongs_to_collection_bin"]/df_genre_2["Count"]
df_genre_3["belongs_to_collection_bin"] = df_genre_3["belongs_to_collection_bin"]/df_genre_3["Count"]

df_genre_1.fillna(0,inplace=True)
df_genre_2.fillna(0,inplace=True)
df_genre_3.fillna(0,inplace=True)

df_genre_1.sort_values(by=["profit (%)"],inplace=True,ignore_index=True)
df_genre_2.sort_values(by=["profit (%)"],inplace=True,ignore_index=True)
df_genre_3.sort_values(by=["profit (%)"],inplace=True,ignore_index=True)


In [245]:
tabela_geral[tabela_geral['Genre 1']=="Family"][["Series_Title","budget",'Gross']]

,Series_Title,budget,Gross
687,E.T. the Extra-Terrestrial,10500000,435110554.0
697,Willy Wonka & the Chocolate Factory,3000000,4000000.0


In [246]:
(((435110554.0+4000000.0)-(10500000+3000000))/(10500000+3000000))*100

3152.6707703703705

In [247]:
df_genre_1.head(20)

,Genre 1,Gross,budget,revenue,profit overall,belongs_to_collection_bin,Count,Gross_mean,budget_mean,profit (%),profit mean
0,Fantasy,0.000000e+00,0,57703,0.000000e+00,0.000000,2,0.000000e+00,0.000000e+00,0.000000,0.000000e+00
1,Mystery,2.739558e+08,166979000,585330992,1.069768e+08,0.000000,12,2.282965e+07,1.391492e+07,64.066026,8.914734e+06
2,Adventure,5.273754e+09,2851765374,14341715170,2.421989e+09,0.236111,72,7.324659e+07,3.960785e+07,84.929463,3.363874e+07
3,Animation,8.573824e+09,4454622386,22130844100,4.119202e+09,0.390244,82,1.045588e+08,5.432466e+07,92.470285,5.023417e+07
4,Crime,3.179785e+09,1626739013,7048171705,1.553046e+09,0.205607,107,2.971761e+07,1.520317e+07,95.469863,1.451444e+07
5,Action,2.001680e+10,9567926517,50477737849,1.044887e+10,0.532164,171,1.170573e+08,5.595279e+07,109.207251,6.110450e+07
6,Biography,4.750170e+09,2215670200,10140064370,2.534500e+09,0.045455,88,5.397920e+07,2.517807e+07,114.389749,2.880113e+07
7,Drama,9.022142e+09,3434244170,19917058751,5.587898e+09,0.104167,288,3.132688e+07,1.192446e+07,162.711147,1.940242e+07
8,Comedy,4.164812e+09,1155924107,8237762095,3.008887e+09,0.141935,155,2.686975e+07,7.457575e+06,260.301470,1.941218e+07
9,Thriller,1.755074e+07,4000000,11000000,1.355074e+07,0.000000,1,1.755074e+07,4.000000e+06,338.768525,1.355074e+07


## Gênero 1, 2, 3

In [248]:

# usando o GPT para auxiliar na estilização do grafico, o deixando mais visual
# Função auxiliar para criar o gráfico de um df_genre
def plot_genre(df_genre, genre_label):
    df_genre = df_genre.replace([float("inf"), -float("inf")], None).dropna()
    fig = go.Figure()

    # Eixo Y1: Count e Profit (%)
    fig.add_trace(go.Scatter(
        x=df_genre[genre_label],
        y=df_genre["Count"],
        mode="lines+markers",
        name="Count",
        line=dict(width=3, color="royalblue"),
        marker=dict(symbol="circle", size=10)
    ))

    fig.add_trace(go.Scatter(
        x=df_genre[genre_label],
        y=df_genre["profit (%)"],
        mode="lines+markers",
        name="Profit (%)",
        line=dict(width=3, color="firebrick", dash="dot"),
        marker=dict(symbol="diamond", size=10)
    ))

    # Eixo Y2: Budget Mean, Gross Mean e Profit Mean
    fig.add_trace(go.Scatter(
        x=df_genre[genre_label],
        y=df_genre["budget_mean"],
        mode="lines+markers",
        name="Budget Mean",
        line=dict(width=3, color="green", dash="dash"),
        marker=dict(symbol="square", size=10),
        yaxis="y2"
    ))

    fig.add_trace(go.Scatter(
        x=df_genre[genre_label],
        y=df_genre["Gross_mean"],
        mode="lines+markers",
        name="Gross Mean",
        line=dict(width=3, color="orange", dash="dashdot"),
        marker=dict(symbol="triangle-up", size=10),
        yaxis="y2"
    ))

    fig.add_trace(go.Scatter(
        x=df_genre[genre_label],
        y=df_genre["profit mean"],
        mode="lines+markers",
        name="Profit Mean",
        line=dict(width=3, color="purple", dash="longdash"),
        marker=dict(symbol="star", size=12),
        yaxis="y2"
    ))

    # Layout com dois eixos Y e hover unificado
    fig.update_layout(
        title=f"Análise de {genre_label} – Count, Profit (%) e Métricas Financeiras",
        xaxis=dict(title="Gênero"),
        yaxis=dict(title="Count / Profit (%)", side="left"),
        yaxis2=dict(title="Budget / Gross / Profit Mean", overlaying="y", side="right"),
        legend=dict(title="Métricas", orientation="h", yanchor="bottom", y=1.02),
        hovermode="x unified",
        template="plotly_white",
        width=1000,
        height=600
    )

    fig.show()

# Plotando os três gráficos na mesma célula
plot_genre(df_genre_1, "Genre 1")
plot_genre(df_genre_2, "Genre 2")
plot_genre(df_genre_3, "Genre 3")


# Analise em Relação aos Atores

In [249]:
tabela_geral.columns

Index(['Series_Title', 'Released_Year', 'Certificate', 'Runtime (min)',
       'Genre', 'Genre 1', 'Genre 2', 'Genre 3', 'IMDB_Rating', 'Overview',
       'Meta_score', 'Director', 'Star1', 'Star2', 'Star3', 'Star4',
       'No_of_Votes', 'Gross', 'id', 'nome_no_dataset', 'data_no_dataset',
       'adult', 'backdrop_path', 'budget', 'homepage', 'imdb_id',
       'origin_country', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_countries', 'release_date',
       'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title',
       'video', 'vote_average', 'vote_count', 'belongs_to_collection_name',
       'genres_names', 'production_companies_names',
       'belongs_to_collection_bin', 'profit overall', 'Count'],
      dtype='object')

In [250]:
# Transformar atores em formato longo
actors_long = pd.melt(
    tabela_geral,
    id_vars=["Gross","budget","revenue","profit overall","belongs_to_collection_bin"],
    value_vars=["Star1","Star2","Star3","Star4"],
    var_name="star_position",
    value_name="actor"
)

# Agrupar por ator
df_actor = actors_long.groupby("actor").agg(
    Count=("Gross", "count"),
    Gross_total=("Gross", "sum"),
    budget_total=("budget", "sum"),
    profit_total=("profit overall", "sum"),
    collection_ratio=("belongs_to_collection_bin", "mean")
).reset_index()

# lucro percentual correto por ator
df_actor["profit (%)"] = 100 * df_actor["profit_total"] / df_actor["budget_total"]
df_actor["profit (%)"] = df_actor["profit (%)"].replace([float("inf"), -float("inf")], 0)

# médias por filme
df_actor["gross_mean"] = df_actor["Gross_total"] / df_actor["Count"]
df_actor["budget_mean"] = df_actor["budget_total"] / df_actor["Count"]
df_actor["profit_mean"] = df_actor["profit_total"] / df_actor["Count"]

# preencher possíveis NaNs
df_actor.fillna(0, inplace=True)

# top 100 atores por número de filmes
df_actor = df_actor.sort_values("Count", ascending=False).head(100)


In [251]:
(8.249446e+07 - 5.800000e+06)/5.800000e+06

13.22318275862069

In [252]:
# Ordenar pelo Profit (%)
df_actor_sorted = df_actor.sort_values("profit (%)", ascending=False).head(100)

fig = go.Figure()

# Eixo Y1: Count
fig.add_trace(go.Scatter(
    x=df_actor_sorted["actor"],
    y=df_actor_sorted["Count"],
    mode="lines+markers",
    name="Count",
    line=dict(width=3, color="royalblue"),
    marker=dict(symbol="circle", size=8, color=df_actor_sorted["collection_ratio"], colorscale="Viridis", showscale=True, colorbar=dict(title="Franquia (média)")),
    yaxis="y1"
))

# Eixo Y2: Profit (%)
fig.add_trace(go.Scatter(
    x=df_actor_sorted["actor"],
    y=df_actor_sorted["profit (%)"],
    mode="lines+markers",
    name="Profit (%)",
    line=dict(width=3, color="firebrick", dash="dot"),
    marker=dict(symbol="diamond", size=8, color=df_actor_sorted["collection_ratio"], colorscale="Viridis"),
    yaxis="y1"
))

# Eixo Y2 adicional: Profit absoluto (profit_mean)
fig.add_trace(go.Scatter(
    x=df_actor_sorted["actor"],
    y=df_actor_sorted["profit_mean"],
    mode="lines+markers",
    name="Profit Médio",
    line=dict(width=3, color="darkgreen"),
    marker=dict(symbol="square", size=8, color=df_actor_sorted["collection_ratio"], colorscale="Viridis"),
    yaxis="y2"
))

# Eixo Y2 adicional: Profit absoluto (profit_mean)
fig.add_trace(go.Scatter(
    x=df_actor_sorted["actor"],
    y=df_actor_sorted["budget_mean"],
    mode="lines+markers",
    name="Budget Médio",
    line=dict(width=3, color="red"),
    marker=dict(symbol="square", size=8, color=df_actor_sorted["collection_ratio"], colorscale="Viridis"),
    yaxis="y2"
))
# Layout
fig.update_layout(
    title="Top 100 Atores – Count, Profit (%) e Profit Bruto (ordenado por Profit %)",
    xaxis=dict(title="Atores", tickangle=-45),
    yaxis=dict(title="Profit (%) / Count", side="left"),
    yaxis2=dict(title="Profit Bruto / Budget Médio", overlaying="y", side="right"),
    legend=dict(title="Métricas", orientation="h", yanchor="bottom", y=1.02),
    hovermode="x unified",
    template="plotly_white",
    width=1200,
    height=700
)

fig.show()


In [253]:
# Filtrar filmes em que Robert Downey Jr aparece
rdj_films = tabela_geral[
    (tabela_geral['Star1'] == 'Robert Downey Jr.') |
    (tabela_geral['Star2'] == 'Robert Downey Jr.') |
    (tabela_geral['Star3'] == 'Robert Downey Jr.') |
    (tabela_geral['Star4'] == 'Robert Downey Jr.')
]
rdj_films['profit (%)'] = 100*(rdj_films['profit overall']/rdj_films["budget"])
# Mostrar colunas relevantes
rdj_films[['Series_Title', 'Released_Year', 'Star1', 'Star2', 'Star3', 'Star4', 'Gross', 'budget', 'profit overall', 'profit (%)']]


/tmp/ipython-input-3577416383.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Series_Title,Released_Year,Star1,Star2,Star3,Star4,Gross,budget,profit overall,profit (%)
58,Avengers: Endgame,2019.0,Joe Russo,Robert Downey Jr.,Chris Evans,Mark Ruffalo,858373000.0,356000000,502373000.0,141.116011
59,Avengers: Infinity War,2018.0,Joe Russo,Robert Downey Jr.,Chris Hemsworth,Mark Ruffalo,678815482.0,300000000,378815482.0,126.271827
356,The Avengers,2012.0,Robert Downey Jr.,Chris Evans,Scarlett Johansson,Jeremy Renner,623279547.0,220000000,403279547.0,183.308885
501,Iron Man,2008.0,Robert Downey Jr.,Gwyneth Paltrow,Terrence Howard,Jeff Bridges,318412101.0,140000000,178412101.0,127.437215
582,Captain America: Civil War,2016.0,Joe Russo,Chris Evans,Robert Downey Jr.,Scarlett Johansson,408084349.0,250000000,158084349.0,63.233740
766,Zodiac,2007.0,Jake Gyllenhaal,Robert Downey Jr.,Mark Ruffalo,Anthony Edwards,33080084.0,65000000,-31919916.0,-49.107563
913,Sherlock Holmes,2009.0,Robert Downey Jr.,Jude Law,Rachel McAdams,Mark Strong,209028679.0,90000000,119028679.0,132.254088


a bilheteria é dos EUA e Candá pelo visto

# Análise em Relação ao Diretor

In [254]:
tabela_geral.columns

Index(['Series_Title', 'Released_Year', 'Certificate', 'Runtime (min)',
       'Genre', 'Genre 1', 'Genre 2', 'Genre 3', 'IMDB_Rating', 'Overview',
       'Meta_score', 'Director', 'Star1', 'Star2', 'Star3', 'Star4',
       'No_of_Votes', 'Gross', 'id', 'nome_no_dataset', 'data_no_dataset',
       'adult', 'backdrop_path', 'budget', 'homepage', 'imdb_id',
       'origin_country', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_countries', 'release_date',
       'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title',
       'video', 'vote_average', 'vote_count', 'belongs_to_collection_name',
       'genres_names', 'production_companies_names',
       'belongs_to_collection_bin', 'profit overall', 'Count'],
      dtype='object')

In [255]:
# Agrupar por diretor
tabela_diretor = tabela_geral.groupby("Director")[["IMDB_Rating", "Meta_score", "profit overall", "budget", "Runtime (min)", "Count", "No_of_Votes", "popularity"]].sum().reset_index()

# Profit percentual
tabela_diretor["profit (%)"] = 100 * tabela_diretor["profit overall"] / tabela_diretor["budget"]
tabela_diretor["profit (%)"] = tabela_diretor["profit (%)"].replace([float("inf"), -float("inf")], 0)

# Média de métricas por filme
tabela_diretor["Runtime (min)"] = tabela_diretor["Runtime (min)"] / tabela_diretor["Count"]
tabela_diretor["profit mean"] = tabela_diretor["profit overall"] / tabela_diretor["Count"]
tabela_diretor["No_of_Votes"] = tabela_diretor["No_of_Votes"] / tabela_diretor["Count"]
tabela_diretor["popularity"] = tabela_diretor["popularity"] / tabela_diretor["Count"]
tabela_diretor["budget_mean"] = tabela_diretor["budget"] / tabela_diretor["Count"]

# Escala do Meta_score para 0-10 como IMDB
tabela_diretor["Meta_score"] = tabela_diretor["Meta_score"] / tabela_diretor["Count"]
tabela_diretor["IMDB_Rating"] = tabela_diretor["IMDB_Rating"] / tabela_diretor["Count"]

# Substituir NaNs e ordenar pelo lucro percentual
tabela_diretor.fillna(0, inplace=True)
tabela_diretor.sort_values(by="profit mean", ascending=False, inplace=True, ignore_index=True)


In [256]:

# Ordenar pelos diretores com maior lucro percentual
df_diretor_sorted = tabela_diretor.sort_values("profit mean", ascending=False).head(50)

fig_finance = go.Figure()

# Eixo Y1: Budget e Profit Mean
fig_finance.add_trace(go.Bar(
    x=df_diretor_sorted["Director"],
    y=df_diretor_sorted["budget_mean"],
    name="Budget Mean",
    marker_color="lightblue",
))

fig_finance.add_trace(go.Bar(
    x=df_diretor_sorted["Director"],
    y=df_diretor_sorted["profit mean"],
    name="Profit Mean",
    marker_color="royalblue",
))

# Eixo Y2: Profit (%)
fig_finance.add_trace(go.Scatter(
    x=df_diretor_sorted["Director"],
    y=df_diretor_sorted["profit (%)"],
    mode="lines+markers",
    name="Profit (%)",
    line=dict(color="firebrick", width=3, dash="dot"),
    marker=dict(symbol="diamond", size=8),
    yaxis="y2"
))


# Eixo Y2: Profit (%)
fig_finance.add_trace(go.Scatter(
    x=df_diretor_sorted["Director"],
    y=df_diretor_sorted["Count"],
    mode="lines+markers",
    name="Count",
    line=dict(color="darkgreen", width=3, dash="dot"),
    marker=dict(symbol="star", size=8),
    yaxis="y2"
))


# Layout
fig_finance.update_layout(
    title="Top 50 Diretores – Análise Financeira",
    xaxis=dict(title="Diretor", tickangle=-45),
    yaxis=dict(title="Valores em USD", side="left"),
    yaxis2=dict(title="Profit (%) / Count", overlaying="y", side="right"),
    barmode="group",
    template="plotly_white",
    width=1200,
    height=700,
    legend=dict(title="Métricas", orientation="h", yanchor="bottom", y=1.02),
    hovermode="x unified"
)

fig_finance.show()


In [257]:
fig_quality = go.Figure()

df_diretor_sorted = tabela_diretor.sort_values("IMDB_Rating", ascending=False).head(50)


# Eixo Y1: IMDB, Meta_score e Runtime
fig_quality.add_trace(go.Bar(
    x=df_diretor_sorted["Director"],
    y=df_diretor_sorted["IMDB_Rating"]*10,
    name="IMDB Rating",
    marker_color="lightgreen",
))

fig_quality.add_trace(go.Bar(
    x=df_diretor_sorted["Director"],
    y=df_diretor_sorted["Meta_score"],
    name="Meta Score (0-100)",
    marker_color="green",
))

fig_quality.add_trace(go.Bar(
    x=df_diretor_sorted["Director"],
    y=df_diretor_sorted["Runtime (min)"],
    name="Runtime (min)",
    marker_color="darkgreen",
))

# Eixo Y2: Count, No_of_Votes e Popularidade
fig_quality.add_trace(go.Scatter(
    x=df_diretor_sorted["Director"],
    y=df_diretor_sorted["Count"],
    mode="lines+markers",
    name="Count",
    line=dict(color="blue", width=3, dash="dot"),
    marker=dict(symbol="circle", size=8),
    yaxis="y2"
))

fig_quality.add_trace(go.Scatter(
    x=df_diretor_sorted["Director"],
    y=df_diretor_sorted["No_of_Votes"],
    mode="lines+markers",
    name="Número de Votos",
    line=dict(color="orange", width=3, dash="dot"),
    marker=dict(symbol="diamond", size=8),
    yaxis="y2"
))

fig_quality.add_trace(go.Scatter(
    x=df_diretor_sorted["Director"],
    y=df_diretor_sorted["popularity"],
    mode="lines+markers",
    name="Popularidade",
    line=dict(color="red", width=3, dash="dash"),
    marker=dict(symbol="cross", size=8),
    yaxis="y2"
))

# Layout
fig_quality.update_layout(
    title="Top 50 Diretores – Qualidade, Duração e Métricas Populares",
    xaxis=dict(title="Diretor", tickangle=-45),
    yaxis=dict(title="IMDB, Meta Score, Runtime", side="left"),
    yaxis2=dict(title="Count / Votos / Popularidade", overlaying="y", side="right"),
    barmode="group",
    template="plotly_white",
    width=1200,
    height=700,
    legend=dict(title="Métricas", orientation="h", yanchor="bottom", y=1.02),
    hovermode="x unified"
)

fig_quality.show()


# Análise de Duração

In [258]:
tabela_duracao = tabela_geral.groupby(["Released_Year"])[["Runtime (min)","Count","profit overall","budget"]].sum().reset_index()

# Profit percentual
tabela_duracao["profit (%)"] = 100 * tabela_duracao["profit overall"] / tabela_duracao["budget"]
tabela_duracao["profit (%)"] = tabela_duracao["profit (%)"].replace([float("inf"), -float("inf")], 0)

# Média de duração e lucro por filme
tabela_duracao["Runtime (min)"] = tabela_duracao["Runtime (min)"] / tabela_duracao["Count"]
tabela_duracao["profit mean"] = tabela_duracao["profit overall"] / tabela_duracao["Count"]
tabela_duracao["budget mean"] = tabela_duracao["budget"] / tabela_duracao["Count"]



tabela_duracao.fillna(0, inplace=True)
tabela_duracao.sort_values(by=["Released_Year"], inplace=True, ignore_index=True)


In [259]:

# Selecionar tabela já agregada
tabela_duracao_sorted = tabela_duracao.sort_values("Released_Year")

fig = go.Figure()

# Eixo Y1: Runtime médio
fig.add_trace(go.Scatter(
    x=tabela_duracao_sorted["Released_Year"],
    y=tabela_duracao_sorted["Runtime (min)"],
    mode="lines+markers",
    name="Duração Média (min)",
    line=dict(color="royalblue", width=3),
    marker=dict(symbol="circle", size=6),
    yaxis="y1"
))

fig.add_trace(go.Scatter(
    x=tabela_duracao_sorted["Released_Year"],
    y=tabela_duracao_sorted["Count"],
    mode="lines+markers",
    name="Quantidade de Filmes",
    line=dict(color="brown", width=3),
    marker=dict(symbol="circle", size=6),
    yaxis="y1"
))

# Eixo Y2: Profit (%)
fig.add_trace(go.Scatter(
    x=tabela_duracao_sorted["Released_Year"],
    y=tabela_duracao_sorted["profit (%)"],
    mode="lines+markers",
    name="Profit (%)",
    line=dict(color="firebrick", width=3, dash="dot"),
    marker=dict(symbol="diamond", size=6),
    yaxis="y2"
))

# Eixo Y2: Profit médio por filme
fig.add_trace(go.Scatter(
    x=tabela_duracao_sorted["Released_Year"],
    y=tabela_duracao_sorted["profit mean"],
    mode="lines+markers",
    name="Profit Médio por Filme",
    line=dict(color="green", width=3, dash="dash"),
    marker=dict(symbol="square", size=6),
    yaxis="y2"
))

# Eixo Y2: Profit médio por filme
fig.add_trace(go.Scatter(
    x=tabela_duracao_sorted["Released_Year"],
    y=tabela_duracao_sorted["budget mean"],
    mode="lines+markers",
    name="Budget Médio por Filme",
    line=dict(color="yellow", width=3, dash="dash"),
    marker=dict(symbol="square", size=6),
    yaxis="y2"
))

# Layout
fig.update_layout(
    title="Evolução da Duração e Lucro dos Filmes ao Longo dos Anos",
    xaxis=dict(title="Ano de Lançamento"),
    yaxis=dict(title="Duração Média (min)", side="left"),
    yaxis2=dict(title="Lucro e Profit (%)", overlaying="y", side="right"),
    legend=dict(title="Métricas", orientation="h", yanchor="bottom", y=1.02),
    hovermode="x unified",
    template="plotly_white",
    width=1200,
    height=700
)

fig.show()


interessante vermos os filmes apenas em streaming tem nenhum lucro direto, e tambem pela pandemia filmes de 2020 nao tem bom lucro no caso é ate negativo, muitos foram direto para o streaming

In [260]:
tabela_geral[tabela_geral["Released_Year"]==2020]

,Series_Title,Released_Year,Certificate,Runtime (min),Genre,Genre 1,Genre 2,Genre 3,IMDB_Rating,Overview,...,title,video,vote_average,vote_count,belongs_to_collection_name,genres_names,production_companies_names,belongs_to_collection_bin,profit overall,Count
17,Hamilton,2020.0,PG-13,160.0,"Biography, Drama, History",Biography,Drama,History,8.6,The real life of one of America's foremost fou...,...,The Undefeated Presents: Hamilton In-Depth,False,7.000,2,0,['Documentary'],[],0,0.0,1
19,Soorarai Pottru,2020.0,U,153.0,Drama,Drama,0,0,8.6,"Nedumaaran Rajangam ""Maara"" sets out to make t...",...,Soorarai Pottru,False,7.725,100,0,"['Drama', 'Action']","['2D Entertainment', 'Sikhya Entertainment']",0,0.0,1
204,Soul,2020.0,U,100.0,"Animation, Adventure, Comedy",Animation,Adventure,Comedy,8.1,"After landing the gig of a lifetime, a New Yor...",...,Soul,False,8.104,10907,0,"['Animation', 'Family', 'Comedy', 'Fantasy']",['Pixar'],0,-150000000.0,1
463,Dil Bechara,2020.0,UA,101.0,"Comedy, Drama, Romance",Comedy,Drama,Romance,7.9,The emotional journey of two hopelessly in lov...,...,Dil Bechara,False,6.604,106,0,"['Drama', 'Romance']",['Fox Star Studios'],0,0.0,1
611,The Trial of the Chicago 7,2020.0,R,129.0,"Drama, History, Thriller",Drama,History,Thriller,7.8,The story of 7 people on trial stemming from v...,...,The Trial of the Chicago 7,False,7.678,3167,0,"['Drama', 'History']","['DreamWorks Pictures', 'Marc Platt Productions']",0,-35000000.0,1
612,Druk,2020.0,0,117.0,"Comedy, Drama",Comedy,Drama,0,7.8,"Four friends, all high school teachers, test a...",...,Another Round,False,7.636,3515,0,"['Comedy', 'Drama']","['Zentropa Entertainments', 'Film i Väst', 'To...",0,-5150000.0,1


# Lucro em relação ao país de origem e lingua falada principal

In [261]:
tabela_geral.columns

Index(['Series_Title', 'Released_Year', 'Certificate', 'Runtime (min)',
       'Genre', 'Genre 1', 'Genre 2', 'Genre 3', 'IMDB_Rating', 'Overview',
       'Meta_score', 'Director', 'Star1', 'Star2', 'Star3', 'Star4',
       'No_of_Votes', 'Gross', 'id', 'nome_no_dataset', 'data_no_dataset',
       'adult', 'backdrop_path', 'budget', 'homepage', 'imdb_id',
       'origin_country', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_countries', 'release_date',
       'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title',
       'video', 'vote_average', 'vote_count', 'belongs_to_collection_name',
       'genres_names', 'production_companies_names',
       'belongs_to_collection_bin', 'profit overall', 'Count'],
      dtype='object')

In [262]:
tabela_linguas = tabela_geral.groupby(['original_language'])[["IMDB_Rating","Meta_score",'profit overall',"budget","Runtime (min)","Count",'No_of_Votes']].sum().reset_index()

# Profit percentual
tabela_linguas["profit (%)"] = 100 *tabela_linguas["profit overall"] / tabela_linguas["budget"]
tabela_linguas["profit (%)"] = tabela_linguas["profit (%)"].replace([float("inf"), -float("inf")], 0)

# Média de duração e lucro por filme
tabela_linguas["Runtime (min)"] = tabela_linguas["Runtime (min)"] / tabela_linguas["Count"]
tabela_linguas["profit mean"] = tabela_linguas["profit overall"] / tabela_linguas["Count"]
tabela_linguas['No_of_Votes'] = tabela_linguas['No_of_Votes'] / tabela_linguas["Count"]
# pra ficar na mesma escala do imdb
tabela_linguas['Meta_score'] = (tabela_linguas['Meta_score'] / tabela_linguas["Count"])
tabela_linguas['IMDB_Rating'] = tabela_linguas['IMDB_Rating'] / tabela_linguas["Count"]


tabela_linguas.fillna(0, inplace=True)
tabela_linguas.sort_values(by=["original_language"], inplace=True, ignore_index=True)

In [263]:
tabela_linguas

,original_language,IMDB_Rating,Meta_score,profit overall,budget,Runtime (min),Count,No_of_Votes,profit (%),profit mean
0,ar,8.400000,75.000000,-2.338904e+06,4000000,126.000000,1,62635.000000,-58.472600,-2.338904e+06
1,bs,7.900000,84.000000,4.767700e+04,1012153,98.000000,1,44618.000000,4.710454,4.767700e+04
2,cn,7.900000,65.875000,-2.514751e+07,45760000,105.500000,8,95349.000000,-54.955214,-3.143438e+06
3,da,7.850000,73.125000,-3.328790e+07,41520773,116.000000,8,76008.875000,-80.171687,-4.160988e+06
4,de,7.928571,51.619048,-3.224204e+07,86717450,121.047619,21,116127.238095,-37.180567,-1.535335e+06
5,en,7.924677,73.476327,3.102936e+10,24117522471,121.657102,697,344016.727403,128.658982,4.451845e+07
6,es,7.894737,56.315789,2.568604e+07,82922296,118.842105,19,129402.000000,30.976032,1.351897e+06
7,et,8.200000,73.000000,1.445010e+05,0,87.000000,1,40382.000000,0.000000,1.445010e+05
8,fa,8.100000,86.000000,5.361154e+06,5180000,113.750000,4,95596.500000,103.497181,1.340288e+06
9,fr,7.936957,59.260870,-9.579419e+07,357687705,110.695652,46,124553.108696,-26.781515,-2.082482e+06


In [264]:
fig1 = go.Figure()

# Eixo Y1: IMDB_Rating e Meta_score (dividido por 100)
fig1.add_trace(go.Scatter(
    x=tabela_linguas["original_language"],
    y=tabela_linguas["IMDB_Rating"],
    mode="lines+markers",
    name="IMDB Rating",
    line=dict(color="royalblue", width=3),
    yaxis="y1"
))
fig1.add_trace(go.Scatter(
    x=tabela_linguas["original_language"],
    y=tabela_linguas["Meta_score"]/10,
    mode="lines+markers",
    name="Meta Score (escala 0-10)",
    line=dict(color="firebrick", width=3, dash="dot"),
    yaxis="y1"
))

# Eixo Y2: No_of_Votes, Runtime e Count
fig1.add_trace(go.Scatter(
    x=tabela_linguas["original_language"],
    y=tabela_linguas["No_of_Votes"],
    mode="lines+markers",
    name="Número de Votos",
    line=dict(color="green", width=3),
    yaxis="y2"
))
fig1.add_trace(go.Scatter(
    x=tabela_linguas["original_language"],
    y=tabela_linguas["Runtime (min)"],
    mode="lines+markers",
    name="Runtime Médio",
    line=dict(color="orange", width=3, dash="dot"),
    yaxis="y2"
))
fig1.add_trace(go.Scatter(
    x=tabela_linguas["original_language"],
    y=tabela_linguas["Count"],
    mode="lines+markers",
    name="Número de Filmes",
    line=dict(color="purple", width=3, dash="dash"),
    yaxis="y2"
))

# Layout
fig1.update_layout(
    title="Idiomas – Avaliações, Count e Runtime Médio",
    xaxis=dict(title="Idioma"),
    yaxis=dict(title="Avaliações (IMDB e Meta)", side="left"),
    yaxis2=dict(title="Votos / Runtime / Count", overlaying="y", side="right"),
    legend=dict(title="Métricas", orientation="h", yanchor="bottom", y=1.02),
    hovermode="x unified",
    template="plotly_white",
    width=1200,
    height=700
)

fig1.show()


In [265]:
fig2 = go.Figure()

# Profit (%)
fig2.add_trace(go.Scatter(
    x=tabela_linguas["original_language"],
    y=tabela_linguas["profit (%)"],
    mode="lines+markers",
    name="Profit (%)",
    line=dict(color="firebrick", width=3),
    yaxis="y1"
))

# Profit mean
fig2.add_trace(go.Scatter(
    x=tabela_linguas["original_language"],
    y=tabela_linguas["profit mean"],
    mode="lines+markers",
    name="Profit Médio",
    line=dict(color="royalblue", width=3, dash="dot"),
    yaxis="y2"
))

# Layout
fig2.update_layout(
    title="Idiomas – Lucro (%) e Lucro Médio",
    xaxis=dict(title="Idioma"),
    yaxis=dict(title="Profit (%)", side="left"),
    yaxis2=dict(title="Profit Médio", overlaying="y", side="right"),
    legend=dict(title="Métricas", orientation="h", yanchor="bottom", y=1.02),
    hovermode="x unified",
    template="plotly_white",
    width=1200,
    height=700
)

fig2.show()


# Lucro e Métricas de Avaliação IMDb

In [266]:
# Agrupar por Certificate
tabela_nota = tabela_geral.groupby(["IMDB_Rating"])[["Meta_score",'profit overall',"budget","Count","No_of_Votes","Runtime (min)"]].sum().reset_index()

# Profit percentual
tabela_nota["profit (%)"] = 100 * tabela_nota["profit overall"] / tabela_nota["budget"]
tabela_nota["profit (%)"] = tabela_nota["profit (%)"].replace([float("inf"), -float("inf")], 0)

# Média de duração e lucro por filme
tabela_nota["Runtime (min)"] = tabela_nota["Runtime (min)"] / tabela_nota["Count"]
tabela_nota["profit mean"] = tabela_nota["profit overall"] / tabela_nota["Count"]
tabela_nota['No_of_Votes'] = tabela_nota['No_of_Votes'] / tabela_nota["Count"]
# Escala do Meta_score para 0-10 como IMDB
tabela_nota['Meta_score'] = tabela_nota['Meta_score'] / tabela_nota["Count"]

# Substituir NaNs e ordenar
tabela_nota.fillna(0, inplace=True)


tabela_nota.sort_values("IMDB_Rating", ascending=True,inplace=True)

In [267]:
fig2 = go.Figure()

# Profit (%)
fig2.add_trace(go.Scatter(
    x=tabela_nota["IMDB_Rating"],
    y=tabela_nota["profit (%)"],
    mode="lines+markers",
    name="Profit (%)",
    line=dict(color="firebrick", width=3),
    yaxis="y1"
))

# Profit mean
fig2.add_trace(go.Scatter(
    x=tabela_nota["IMDB_Rating"],
    y=tabela_nota["profit mean"],
    mode="lines+markers",
    name="Profit Médio",
    line=dict(color="royalblue", width=3, dash="dot"),
    yaxis="y2"
))


# Profit mean
fig2.add_trace(go.Scatter(
    x=tabela_nota["IMDB_Rating"],
    y=tabela_nota["Count"],
    mode="lines+markers",
    name="Número de Filmes",
    line=dict(color="black", width=3, dash="dot"),
    yaxis="y1"
))


# Layout
fig2.update_layout(
    title="Notas IMDb – Lucro (%) e Lucro Médio",
    xaxis=dict(title="IMDB Rating"),
    yaxis=dict(title="Profit (%)", side="left"),
    yaxis2=dict(title="Profit Médio", overlaying="y", side="right"),
    legend=dict(title="Métricas", orientation="h", yanchor="bottom", y=1.02),
    hovermode="x unified",
    template="plotly_white",
    width=1200,
    height=700
)

fig2.show()


# Lucro e Métricas Avaliação Metascore

In [268]:
# Agrupar por Certificate
tabela_nota_ms = tabela_geral.groupby(["Meta_score"])[["IMDB_Rating",'profit overall',"budget","Count","No_of_Votes","Runtime (min)"]].sum().reset_index()

# Profit percentual
tabela_nota_ms["profit (%)"] = 100 * tabela_nota_ms["profit overall"] / tabela_nota_ms["budget"]
tabela_nota_ms["profit (%)"] = tabela_nota_ms["profit (%)"].replace([float("inf"), -float("inf")], 0)

# Média de duração e lucro por filme
tabela_nota_ms["Runtime (min)"] = tabela_nota_ms["Runtime (min)"] / tabela_nota_ms["Count"]
tabela_nota_ms["profit mean"] = tabela_nota_ms["profit overall"] / tabela_nota_ms["Count"]
tabela_nota_ms['No_of_Votes'] = tabela_nota_ms['No_of_Votes'] / tabela_nota_ms["Count"]
# Escala do Meta_score para 0-10 como IMDB

# Substituir NaNs e ordenar
tabela_nota_ms.fillna(0, inplace=True)


tabela_nota_ms.sort_values("Meta_score", ascending=True,inplace=True)

In [269]:
fig2 = go.Figure()

# Profit (%)
fig2.add_trace(go.Scatter(
    x=tabela_nota["Meta_score"],
    y=tabela_nota["profit (%)"],
    mode="lines+markers",
    name="Profit (%)",
    line=dict(color="firebrick", width=3),
    yaxis="y1"
))

# Profit mean
fig2.add_trace(go.Scatter(
    x=tabela_nota["Meta_score"],
    y=tabela_nota["profit mean"],
    mode="lines+markers",
    name="Profit Médio",
    line=dict(color="royalblue", width=3, dash="dot"),
    yaxis="y2"
))


# Profit mean
fig2.add_trace(go.Scatter(
    x=tabela_nota["Meta_score"],
    y=tabela_nota["Count"],
    mode="lines+markers",
    name="Número de Filmes",
    line=dict(color="black", width=3, dash="dot"),
    yaxis="y1"
))


# Layout
fig2.update_layout(
    title="Notas Meta Score – Lucro (%) e Lucro Médio",
    xaxis=dict(title="Meta Score"),
    yaxis=dict(title="Profit (%)", side="left"),
    yaxis2=dict(title="Profit Médio", overlaying="y", side="right"),
    legend=dict(title="Métricas", orientation="h", yanchor="bottom", y=1.02),
    hovermode="x unified",
    template="plotly_white",
    width=1200,
    height=700
)

fig2.show()


percebemos que a nota do imdb tem alguma relação com o lucro tanto bruto quanto percentual, o que faz sentido, pois esta é feita atraves de voto popular, então ok. Já para o meta score isso não é verdade não consiguimos extrair informação pois nao há muito um padrao, indicando que o lucro de um filme nao esta atrelado a opiniao dos criticos

# Lucro em Relação a classificação indicativa

interessante por pensar em filmes como deadpool que fez muito dinheiro, mesmo com classificação para maiores

In [270]:
tabela_geral.columns

Index(['Series_Title', 'Released_Year', 'Certificate', 'Runtime (min)',
       'Genre', 'Genre 1', 'Genre 2', 'Genre 3', 'IMDB_Rating', 'Overview',
       'Meta_score', 'Director', 'Star1', 'Star2', 'Star3', 'Star4',
       'No_of_Votes', 'Gross', 'id', 'nome_no_dataset', 'data_no_dataset',
       'adult', 'backdrop_path', 'budget', 'homepage', 'imdb_id',
       'origin_country', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_countries', 'release_date',
       'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title',
       'video', 'vote_average', 'vote_count', 'belongs_to_collection_name',
       'genres_names', 'production_companies_names',
       'belongs_to_collection_bin', 'profit overall', 'Count'],
      dtype='object')

In [271]:
# Agrupar por Certificate
tabela_certificado = tabela_geral.groupby(["Certificate"])[["IMDB_Rating","Meta_score",'profit overall',"budget","Runtime (min)","Count","No_of_Votes"]].sum().reset_index()

# Profit percentual
tabela_certificado["profit (%)"] = 100 * tabela_certificado["profit overall"] / tabela_certificado["budget"]
tabela_certificado["profit (%)"] = tabela_certificado["profit (%)"].replace([float("inf"), -float("inf")], 0)

# Média de duração e lucro por filme
tabela_certificado["Runtime (min)"] = tabela_certificado["Runtime (min)"] / tabela_certificado["Count"]
tabela_certificado["profit mean"] = tabela_certificado["profit overall"] / tabela_certificado["Count"]
tabela_certificado['No_of_Votes'] = tabela_certificado['No_of_Votes'] / tabela_certificado["Count"]
# Escala do Meta_score para 0-10 como IMDB
tabela_certificado['Meta_score'] = tabela_certificado['Meta_score'] / tabela_certificado["Count"]
tabela_certificado['IMDB_Rating'] = tabela_certificado['IMDB_Rating'] / tabela_certificado["Count"]

# Substituir NaNs e ordenar
tabela_certificado.fillna(0, inplace=True)



In [272]:
tabela_certificado

,Certificate,IMDB_Rating,Meta_score,profit overall,budget,Runtime (min),Count,No_of_Votes,profit (%),profit mean
0,0,7.935000,46.090000,-2.167418e+06,170313257,111.150000,100,58294.730000,-1.272607,-2.167418e+04
1,16,8.100000,78.000000,0.000000e+00,0,130.000000,1,47708.000000,0.000000,0.000000e+00
2,A,7.992347,73.964286,6.692024e+09,4779318034,128.872449,196,418445.173469,140.020487,3.414298e+07
3,Approved,7.945455,72.000000,2.802312e+07,13670963,111.090909,11,71679.727273,204.982787,2.547556e+06
4,G,8.000000,71.750000,4.087175e+08,87245800,119.666667,12,134312.500000,468.466628,3.405979e+07
5,GP,7.850000,25.000000,-2.582084e+06,4000000,124.500000,2,45386.000000,-64.552100,-1.291042e+06
6,PG,7.927027,67.270270,4.269556e+07,386623000,114.270270,37,101580.567568,11.043202,1.153934e+06
7,PG-13,7.797674,74.325581,6.429374e+08,798029020,116.953488,43,144101.976744,80.565667,1.495203e+07
8,Passed,8.020588,46.205882,4.405910e+07,80958636,106.205882,34,73523.088235,54.421742,1.295856e+06
9,R,7.869863,70.479452,1.292674e+09,2189462222,122.369863,146,212991.869863,59.040694,8.853929e+06


In [273]:
tabela_certificado.Certificate.values

array([0, '16', 'A', 'Approved', 'G', 'GP', 'PG', 'PG-13', 'Passed', 'R',
       'TV-14', 'TV-MA', 'TV-PG', 'U', 'U/A', 'UA', 'Unrated'],
      dtype=object)

In [274]:


# --- Primeiro gráfico: avaliação e produção ---
fig1 = go.Figure()

# Count
fig1.add_trace(go.Bar(
    x=tabela_certificado["Certificate"],
    y=tabela_certificado["Count"],
    name="Count",
    marker_color="royalblue",
    yaxis="y1",
    opacity=0.6
))

# IMDB_Rating e Meta_score (dividido por 10 para mesma escala)
fig1.add_trace(go.Scatter(
    x=tabela_certificado["Certificate"],
    y=tabela_certificado["IMDB_Rating"],
    mode="lines+markers",
    name="IMDB Rating",
    line=dict(width=3, color="firebrick"),
    yaxis="y2"
))

fig1.add_trace(go.Scatter(
    x=tabela_certificado["Certificate"],
    y=tabela_certificado["Meta_score"]/10,
    mode="lines+markers",
    name="Meta Score (esc. 0-10)",
    line=dict(width=3, color="darkorange", dash="dot"),
    yaxis="y2"
))

# Runtime
fig1.add_trace(go.Scatter(
    x=tabela_certificado["Certificate"],
    y=tabela_certificado["Runtime (min)"],
    mode="lines+markers",
    name="Runtime (min)",
    line=dict(width=3, color="green", dash="dash"),
    yaxis="y2"
))

fig1.update_layout(
    title="Classificação Indicativa – Count, IMDB, Meta Score e Runtime",
    xaxis=dict(title="Certificate"),
    yaxis=dict(title="Count", side="left"),
    yaxis2=dict(title="Média / Escala", overlaying="y", side="right"),
    legend=dict(title="Métricas", orientation="h", yanchor="bottom", y=1.02),
    hovermode="x unified",
    template="plotly_white",
    width=1200,
    height=700
)

fig1.show()


# --- Segundo gráfico: métricas financeiras ---
fig2 = go.Figure()

# Profit (%)
fig2.add_trace(go.Bar(
    x=tabela_certificado["Certificate"],
    y=tabela_certificado["profit (%)"],
    name="Profit (%)",
    marker_color="firebrick",
    opacity=0.6
))

# Profit mean
fig2.add_trace(go.Scatter(
    x=tabela_certificado["Certificate"],
    y=tabela_certificado["profit mean"],
    mode="lines+markers",
    name="Profit Mean (bruto)",
    line=dict(width=3, color="royalblue", dash="dot"),
    yaxis="y2"
))

fig2.update_layout(
    title="Classificação Indicativa – Profit (%) e Profit Mean",
    xaxis=dict(title="Certificate"),
    yaxis=dict(title="Profit (%)", side="left"),
    yaxis2=dict(title="Profit Médio (bruto)", overlaying="y", side="right"),
    legend=dict(title="Métricas", orientation="h", yanchor="bottom", y=1.02),
    hovermode="x unified",
    template="plotly_white",
    width=1200,
    height=700
)

fig2.show()


Esses valores representam a classificação indicativa de filmes e programas, principalmente segundo o padrão dos EUA e algumas variações de TV. Eis um resumo:

G / U / Approved / Passed / A → Livre para todos, conteúdo adequado para todas as idades.

PG / GP / U/A / UA → Orientação dos pais sugerida; algumas cenas podem não ser apropriadas para crianças.

PG-13 / TV-14 → Recomendado para maiores de 13 ou 14 anos; pode conter violência leve, linguagem ou cenas sugestivas.

R / 16 / TV-MA → Conteúdo para maiores de 16–17 anos; violência, nudez ou linguagem forte.

Unrated → Filme sem classificação oficial, pode conter qualquer tipo de conteúdo.

No seu dataset, há também classificações de TV (TV-PG, TV-14, TV-MA) que seguem padrões similares, mas para séries ou programas televisivos.

# Correlação

In [275]:
  #tabela_geral.dtypes


# Criar DataFrame para correlação apenas com colunas numéricas e derivadas
tabela_correlacao = pd.DataFrame()

# Profit percentual (linha a linha)
tabela_correlacao["profit (%)"] = 100 * tabela_geral["profit overall"] / tabela_geral["budget"]
tabela_correlacao["profit (%)"] = tabela_correlacao["profit (%)"].replace([float("inf"), -float("inf")], 0)

# Outras colunas derivadas
tabela_correlacao["franchise_ratio"] = tabela_geral["belongs_to_collection_bin"]
tabela_correlacao["big_budget"] = (tabela_geral["budget"] > 1e7).astype(int)
tabela_correlacao["movie_age"] = 2025 - tabela_geral["Released_Year"]

# Copiar colunas numéricas diretas
colunas_numericas = [
    "Released_Year", "Runtime (min)", "IMDB_Rating", "Meta_score",
    "No_of_Votes", "Gross", "budget", "revenue", "popularity",
    "vote_average", "vote_count", "belongs_to_collection_bin",
    "profit overall"
]

tabela_correlacao = pd.concat([tabela_correlacao, tabela_geral[colunas_numericas]], axis=1)

# Calcular correlação
tabela_correlacao = tabela_correlacao.corr()


In [276]:
tabela_correlacao.IMDB_Rating

,IMDB_Rating
profit (%),0.033367
franchise_ratio,0.020530
big_budget,-0.057999
movie_age,0.132990
Released_Year,-0.132990
Runtime (min),0.242557
IMDB_Rating,1.000000
Meta_score,-0.045607
No_of_Votes,0.479316
Gross,0.084133


In [277]:
# GRAFICOS
# grafico para demostrar as correlações,heatmap
fig = px.imshow(tabela_correlacao.corr())
fig.show()

Lucro absoluto é muito dependente de faturamento e franquias.

Lucro percentual mostra que filmes de baixo orçamento podem ser mais eficientes.

Orçamento alto não garante lucro proporcional.

Popularidade e votos estão mais ligados a faturamento e orçamento do que avaliação crítica.

Um dos principais desafios foi integrar a base original ao TMDb. Como os títulos vinham em formatos diferentes, foi necessário aplicar normalização e fuzzy matching para casar corretamente os filmes. Esse trabalho permitiu enriquecer a análise com variáveis críticas como orçamento, receita global e se o filme pertence a uma franquia, possibilitando insights mais realistas sobre fatores que afetam a bilheteria. Esse passo extra, embora trabalhoso, foi essencial para aproximar a análise de um caso real de indústria.

# Fatores para recomendação

In [278]:
tabela_geral.columns

Index(['Series_Title', 'Released_Year', 'Certificate', 'Runtime (min)',
       'Genre', 'Genre 1', 'Genre 2', 'Genre 3', 'IMDB_Rating', 'Overview',
       'Meta_score', 'Director', 'Star1', 'Star2', 'Star3', 'Star4',
       'No_of_Votes', 'Gross', 'id', 'nome_no_dataset', 'data_no_dataset',
       'adult', 'backdrop_path', 'budget', 'homepage', 'imdb_id',
       'origin_country', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_countries', 'release_date',
       'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title',
       'video', 'vote_average', 'vote_count', 'belongs_to_collection_name',
       'genres_names', 'production_companies_names',
       'belongs_to_collection_bin', 'profit overall', 'Count'],
      dtype='object')

Pontos para recomendação para uma pessoa aleatória



1.   Filme Popular (num of votes e popularity)
2.   Filme que não seja de uma franquia, mais fácil de ver isolado
3.   Tempo, menor que 120 min
4.   Numero de votos ser maior que a media
5.   Melhor Nota (IMDb + MetaScore)/2
6. Classificação Indicativa Geral (G / U )
7. Filme mais atual, depois de 2000, muitas pessoas nao gostam de filmes mais antigos


caso a pessoa já goste de um tipo ou um filme especifico, a abordagem pode ser diferente, como usando o endpoint de recomendação da TMDb



In [279]:
tabela_geral["vote_count"].max()

37877

In [280]:
# calcular média de votos
votes_mean = tabela_geral["No_of_Votes"].mean()
certificados_livres = ["G", "U"]




# aplicar filtros
df_rec = tabela_geral[
    (tabela_geral["belongs_to_collection_bin"] == 0) &
    (tabela_geral["Runtime (min)"] <= 120) &
    (tabela_geral["No_of_Votes"] > votes_mean) &
    (tabela_geral["Certificate"].isin(certificados_livres) &
     (tabela_geral["Released_Year"] >= 2000))
].copy()

# normalizar as metricas de popularidade e fazer a media
popularidade_max = df_rec["popularity"].max()
vote_count_max = df_rec["vote_count"].max()
No_of_Votes_max = df_rec["No_of_Votes"].max()
df_rec["pop_stat"] = ((df_rec["popularity"]/popularidade_max) + (df_rec["vote_count"]/vote_count_max) + (df_rec["No_of_Votes"]/No_of_Votes_max))/3


#multiplica por 10 para deixar na mesma grandeza
df_rec["Media_Notas"] = (df_rec['Meta_score']+df_rec['vote_average']*10+df_rec["IMDB_Rating"]*10)/2

# ordenar por nota IMDb e popularidade
df_rec = df_rec.sort_values(
    by=['pop_stat',"Media_Notas"],
    ascending=[False, False]
)


print("Filme Indicado: ")
# recomendar o primeiro da lista
df_rec[["Series_Title","Released_Year","IMDB_Rating","No_of_Votes","vote_count","Runtime (min)","popularity","Meta_score","vote_average","Media_Notas","Certificate"]].head(5)


Filme Indicado: 


,Series_Title,Released_Year,IMDB_Rating,No_of_Votes,vote_count,Runtime (min),popularity,Meta_score,vote_average,Media_Notas,Certificate
145,Up,2009.0,8.2,935507,20922,96.0,19.4163,88.0,7.958,124.790,U
65,WALL·E,2008.0,8.4,999790,19466,98.0,18.3491,95.0,8.102,130.010,U
366,Ratatouille,2007.0,8.0,641645,17755,111.0,19.4924,96.0,7.831,127.155,U
591,Big Hero 6,2014.0,7.8,410983,16063,102.0,23.1466,74.0,7.726,114.630,U
157,Hauru no ugoku shiro,2004.0,8.2,333915,10487,119.0,16.2938,80.0,8.400,123.000,U


# Inferir Informações do Overview

In [281]:
tabela_geral.columns

Index(['Series_Title', 'Released_Year', 'Certificate', 'Runtime (min)',
       'Genre', 'Genre 1', 'Genre 2', 'Genre 3', 'IMDB_Rating', 'Overview',
       'Meta_score', 'Director', 'Star1', 'Star2', 'Star3', 'Star4',
       'No_of_Votes', 'Gross', 'id', 'nome_no_dataset', 'data_no_dataset',
       'adult', 'backdrop_path', 'budget', 'homepage', 'imdb_id',
       'origin_country', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_countries', 'release_date',
       'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title',
       'video', 'vote_average', 'vote_count', 'belongs_to_collection_name',
       'genres_names', 'production_companies_names',
       'belongs_to_collection_bin', 'profit overall', 'Count'],
      dtype='object')

In [282]:
tabela_geral['overview']

,overview
0,"Spanning the years 1945 to 1955, a chronicle o..."
1,Batman raises the stakes in his war on crime. ...
2,In the continuing saga of the Corleone crime f...
3,The defense and the prosecution have rested an...
4,As armies mass for a final battle that will de...
...,...
993,Holly Golightly is an eccentric New York City ...
994,Wealthy rancher Bick Benedict and dirt-poor co...
995,"In 1941 Hawaii, a private is cruelly punished ..."
996,"During World War II, a small group of survivor..."


In [283]:
tabela_geral['Overview']

,Overview
0,An organized crime dynasty's aging patriarch t...
1,When the menace known as the Joker wreaks havo...
2,The early life and career of Vito Corleone in ...
3,A jury holdout attempts to prevent a miscarria...
4,Gandalf and Aragorn lead the World of Men agai...
...,...
993,A young New York socialite becomes interested ...
994,Sprawling epic covering the life of a Texas ca...
995,"In Hawaii in 1941, a private is cruelly punish..."
996,Several survivors of a torpedoed merchant ship...


In [284]:
# podemos pegar a repetição de palavras por tema sendo assim
# podemos criar um modelo basico

In [285]:
tabela_geral['Overview'] = tabela_geral['Overview'].fillna('')

# iremos nos basear no genero principal

tabela_overview = tabela_geral[tabela_geral['Genre 1'].notna()][['Genre 1',"overview","Overview","tagline"]].copy()

# preencher com string vazia
for col in ["overview","Overview","tagline"]:
    tabela_overview[col] = tabela_overview[col].fillna("").astype(str)


tabela_overview["resumo"] = (tabela_overview["overview"] + " "+ tabela_overview["Overview"] +" "+ tabela_overview["tagline"])

# faremos entao isso usando uma lib, daria para fazer manual tambem, se a palavra aparece ou nao, identificando o final
# dela pelos espaços ou o final da sentença
# isso entao teriam valores binarios, com isso, podemos usar modelos de classificação simples como knn
# poderia fazer na mao a funcao mas daria muito trabalho, poderia ser um topico so para isso
# podemos testar modelos e ver algum aceitavel
# podemos juntar os dois overview, imdb e tmdb e o tagline do tmdb
# limitar palavras que aparecem muito, provalmente tem muitas artigos e tal


vectorizer = TfidfVectorizer(max_features=8000, stop_words='english',min_df=2,lowercase=True,max_df=0.60,token_pattern=r'\b[a-zA-Z]{3,}\b',ngram_range=(1,2) )

X = vectorizer.fit_transform(tabela_overview['Overview'])
y = tabela_overview['Genre 1']

In [286]:
## agora devemos dividir em treino e teste

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)


In [287]:
knn = KNeighborsClassifier(n_neighbors=1)  # testar alguns valores de n
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=1)

In [288]:
y_pred = knn.predict(X_test)
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

      Action       0.38      0.38      0.38        16
   Adventure       0.00      0.00      0.00         8
   Animation       0.38      0.38      0.38         8
   Biography       0.29      0.22      0.25         9
      Comedy       0.21      0.21      0.21        14
       Crime       0.18      0.30      0.22        10
       Drama       0.48      0.41      0.44        34
      Family       0.00      0.00      0.00         0
     Fantasy       0.00      0.00      0.00         0
      Horror       0.50      1.00      0.67         1
     Mystery       0.00      0.00      0.00         0

    accuracy                           0.32       100
   macro avg       0.22      0.26      0.23       100
weighted avg       0.33      0.32      0.32       100



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning:

Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning:

Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning:

Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.



In [289]:


X_train_dense = X_train.toarray()
X_test_dense = X_test.toarray()

nb = GaussianNB()
nb.fit(X_train_dense,y_train)

y_pred = nb.predict(X_test_dense)


print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

      Action       0.17      0.25      0.20        16
   Adventure       0.00      0.00      0.00         8
   Animation       0.33      0.12      0.18         8
   Biography       0.67      0.22      0.33         9
      Comedy       0.33      0.29      0.31        14
       Crime       0.29      0.20      0.24        10
       Drama       0.44      0.62      0.51        34
     Fantasy       0.00      0.00      0.00         0
      Horror       1.00      1.00      1.00         1

    accuracy                           0.35       100
   macro avg       0.36      0.30      0.31       100
weighted avg       0.35      0.35      0.33       100



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning:

Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning:

Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning:

Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.



In [290]:


svm_model = SVC()
svm_model.fit(X_train,y_train)

y_pred = svm_model.predict(X_test)


print('For the Red Wine SVM\n',classification_report(y_test,y_pred))

For the Red Wine SVM
               precision    recall  f1-score   support

      Action       0.75      0.19      0.30        16
   Adventure       0.00      0.00      0.00         8
   Animation       0.00      0.00      0.00         8
   Biography       0.00      0.00      0.00         9
      Comedy       0.00      0.00      0.00        14
       Crime       0.00      0.00      0.00        10
       Drama       0.35      1.00      0.52        34
      Horror       0.00      0.00      0.00         1

    accuracy                           0.37       100
   macro avg       0.14      0.15      0.10       100
weighted avg       0.24      0.37      0.23       100



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



In [291]:


lr_model = LogisticRegression(max_iter=1000, class_weight='balanced')
lr_model.fit(X_train,y_train)
#use the knn model to try to predict the test dataset
y_pred = lr_model.predict(X_test)
## for better understanding of precition and recall
## https://en.wikipedia.org/wiki/Precision_and_recall

print('For the Red Wine LR\n',classification_report(y_test,y_pred))

For the Red Wine LR
               precision    recall  f1-score   support

      Action       0.43      0.56      0.49        16
   Adventure       0.17      0.25      0.20         8
   Animation       0.43      0.38      0.40         8
   Biography       0.40      0.67      0.50         9
      Comedy       0.33      0.21      0.26        14
       Crime       0.25      0.30      0.27        10
       Drama       0.62      0.38      0.47        34
     Fantasy       0.00      0.00      0.00         0
      Horror       1.00      1.00      1.00         1
     Mystery       0.00      0.00      0.00         0

    accuracy                           0.40       100
   macro avg       0.36      0.38      0.36       100
weighted avg       0.44      0.40      0.41       100



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning:

Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning:

Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning:

Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.



In [292]:

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [293]:
## red wine
dtc_model = DecisionTreeClassifier()
dtc_model.fit(X_train,y_train)
#use the knn model to try to predict the test dataset
y_pred = dtc_model.predict(X_test)
## for better understanding of precition and recall
## https://en.wikipedia.org/wiki/Precision_and_recall

print('For the Red Wine DTC\n',classification_report(y_test,y_pred))

For the Red Wine DTC
               precision    recall  f1-score   support

      Action       0.12      0.12      0.12        16
   Adventure       0.00      0.00      0.00         8
   Animation       0.43      0.38      0.40         8
   Biography       0.50      0.22      0.31         9
      Comedy       0.09      0.07      0.08        14
       Crime       0.25      0.20      0.22        10
       Drama       0.42      0.59      0.49        34
   Film-Noir       0.00      0.00      0.00         0
      Horror       0.00      0.00      0.00         1
     Mystery       0.00      0.00      0.00         0
     Western       0.00      0.00      0.00         0

    accuracy                           0.30       100
   macro avg       0.16      0.14      0.15       100
weighted avg       0.28      0.30      0.28       100



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning:

Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning:

Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning:

Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.



In [294]:
## red wine
rfc_model = RandomForestClassifier(class_weight="balanced")
rfc_model.fit(X_train,y_train)
#use the knn model to try to predict the test dataset
y_pred = rfc_model.predict(X_test)
## for better understanding of precition and recall
## https://en.wikipedia.org/wiki/Precision_and_recall

print('For the Red Wine rfc\n',classification_report(y_test,y_pred))


For the Red Wine rfc
               precision    recall  f1-score   support

      Action       0.60      0.38      0.46        16
   Adventure       0.33      0.12      0.18         8
   Animation       1.00      0.12      0.22         8
   Biography       0.75      0.33      0.46         9
      Comedy       0.33      0.07      0.12        14
       Crime       0.43      0.30      0.35        10
       Drama       0.40      0.85      0.55        34
      Horror       0.00      0.00      0.00         1

    accuracy                           0.44       100
   macro avg       0.48      0.27      0.29       100
weighted avg       0.50      0.44      0.39       100



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.




faremos entao isso usando uma lib, daria para fazer manual tambem, se a palavra aparece ou nao, identificando o final
dela pelos espaços ou o final da sentença
isso entao teriam valores binarios, com isso, podemos usar modelos de classificação simples como knn
poderia fazer na mao a funcao mas daria muito trabalho, poderia ser um topico so para isso
podemos testar modelos e ver algum aceitavel
podemos juntar os dois overview, imdb e tmdb e o tagline do tmdb
limitar palavras que aparecem muito, provalmente tem muitas artigos e tal




Para o proposito do exercicio foi até a valida a tentativa obtendo no maximo acc e f1 score medios de 40% mais ou menos no melhor caso, chegamos a um resultado aceitavel, para melhorar este modelo a primeira coisa que eu faria é pegar o maximo de dados possiveis na api do tmdb, pois para este, mesmo que seja melhorado sao poucos dados em relacao ao numero de generos, alem disso tem a questao da multiplicidade de generos em um filme o que atrapalha bem
percebe-se tambem uma variacao ao variar os parametros, pode ser pensado em melhorar isso tambem, mas nao deu tempo
mas o intuito de avaliar as palavras mais frequentes foi bem valido

# Previsão da Nota do IMDb

In [295]:
tabela_geral.columns

Index(['Series_Title', 'Released_Year', 'Certificate', 'Runtime (min)',
       'Genre', 'Genre 1', 'Genre 2', 'Genre 3', 'IMDB_Rating', 'Overview',
       'Meta_score', 'Director', 'Star1', 'Star2', 'Star3', 'Star4',
       'No_of_Votes', 'Gross', 'id', 'nome_no_dataset', 'data_no_dataset',
       'adult', 'backdrop_path', 'budget', 'homepage', 'imdb_id',
       'origin_country', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_countries', 'release_date',
       'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title',
       'video', 'vote_average', 'vote_count', 'belongs_to_collection_name',
       'genres_names', 'production_companies_names',
       'belongs_to_collection_bin', 'profit overall', 'Count'],
      dtype='object')

In [296]:
tabela_geral.IMDB_Rating.unique()

array([9.2, 9. , 8.9, 8.8, 8.7, 8.6, 8.5, 8.4, 8.3, 8.2, 8.1, 8. , 7.9,
       7.8, 7.7, 7.6])

In [297]:
tabela_geral[tabela_geral['Series_Title']=='The Shawshank Redemption']

,Series_Title,Released_Year,Certificate,Runtime (min),Genre,Genre 1,Genre 2,Genre 3,IMDB_Rating,Overview,...,title,video,vote_average,vote_count,belongs_to_collection_name,genres_names,production_companies_names,belongs_to_collection_bin,profit overall,Count


In [298]:
# forca esta a ser um objeto, para podermos obter info dessas colunas


tabela_geral["production_companies_names"] = tabela_geral["production_companies_names"].apply(
    lambda x: ast.literal_eval(x) if isinstance(x, str) and x.startswith("[") else x
)
tabela_geral["production_countries"] = tabela_geral["production_countries"].apply(
    lambda x: ast.literal_eval(x) if isinstance(x, str) and x.startswith("[") else x
)



colunas_modelo = [ 'Released_Year', 'Certificate', 'Runtime (min)',
        'Genre 1', 'Genre 2', 'Genre 3',
       'Meta_score','Star1', 'Star2', 'Star3', 'Star4',
       'No_of_Votes', 'Gross', 'budget','origin_country', 'original_language',
       'popularity', 'production_countries','revenue',
        'vote_average', 'vote_count', 'belongs_to_collection_name',
       'production_companies_names','belongs_to_collection_bin', 'profit overall', 'IMDB_Rating']

tabela_modelo = tabela_geral[colunas_modelo]


# pegar apenas o primeiro valor dessas colunas
for i in range(0,len(tabela_modelo)):
  if tabela_modelo.loc[i,"production_companies_names"] != []:
    #print(tabela_modelo.loc[i,"production_companies_names"])
    tabela_modelo.loc[i,"production_companies_names"] = tabela_modelo.loc[i,"production_companies_names"][0]
  else:
    tabela_modelo.loc[i,"production_companies_names"] = ""

  if tabela_modelo.loc[i,"production_countries"] != []:
    #print(tabela_modelo.loc[i,"production_countries"])
    tabela_modelo.loc[i,"production_countries"] = tabela_modelo.loc[i,"production_countries"][0]['name']
  else:
    tabela_modelo.loc[i,"production_countries"] = ""



o range de notas é pequeno, fazendo com que possamos ate trabalhar com um modelo de classificação, mas tambem podemos usar regressao

In [299]:
print(f"Tamanho: {len(tabela_modelo.IMDB_Rating.unique())}\nMax: {tabela_modelo.IMDB_Rating.max()}\nMin: {tabela_modelo.IMDB_Rating.min()}")


Tamanho: 16
Max: 9.2
Min: 7.6


In [300]:
tipos = tabela_modelo.dtypes
cols_categorical = []

for col in colunas_modelo:
  if tipos[col] == object:
    cols_categorical.append(col)

In [301]:
label_encoders = {}

for col in cols_categorical:
    le = LabelEncoder()
    tabela_modelo.loc[:, col] = le.fit_transform(tabela_modelo[col].astype(str))
    label_encoders[col] = le

In [302]:
def scale_dataset(dataframe,oversample = False):
    X = dataframe[dataframe.columns[:-1]].values
    y = dataframe[dataframe.columns[-1]].values

    scaler = StandardScaler()
    #Scale the number, so we dont have  huge discrepancy between columns, it affects the model
    X = scaler.fit_transform(X)

    #the difference between the len of the values must not be huge, so we have to scale it
    #oversample the one that has the least, taking more of the less class

    if oversample:
        ros = RandomOverSampler(random_state=42)
        X,y = ros.fit_resample(X,y)

    # concat 2 arrays, y is only one dimension so we have to make it 2
    #in this function using -1 is the same as len(y)
    #same as concat in pandas put hstack in numpay


    return X,y

In [303]:
tabela_modelo

,Released_Year,Certificate,Runtime (min),Genre 1,Genre 2,Genre 3,Meta_score,Star1,Star2,Star3,...,popularity,production_countries,revenue,vote_average,vote_count,belongs_to_collection_name,production_companies_names,belongs_to_collection_bin,profit overall,IMDB_Rating
0,1972.0,2,175.0,5,6,0,100.0,417,8,335,...,28.3575,37,245066411,8.686,21784,129,348,1,128966411.0,9.2
1,2008.0,15,152.0,0,5,5,84.0,128,282,1,...,30.4265,36,1004558444,8.523,34320,125,506,1,349858444.0,9.0
2,1974.0,2,202.0,5,6,0,90.0,9,655,703,...,17.1099,37,102600000,8.571,13159,129,348,1,44300000.0,9.0
3,1957.0,13,96.0,5,6,0,96.0,252,463,534,...,12.2818,37,4360000,8.500,9362,0,481,0,3962249.0,9.0
4,2003.0,13,201.0,0,2,5,94.0,184,795,313,...,24.3014,27,1118888979,8.488,25411,143,319,1,283845905.0,8.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
993,1961.0,2,115.0,4,6,14,76.0,50,259,632,...,5.1399,37,9500000,7.600,4312,0,348,0,-2500000.0,7.6
994,1956.0,4,201.0,6,20,0,84.0,185,666,336,...,3.3240,37,32855818,7.614,721,0,506,0,-5400000.0,7.6
995,1953.0,8,118.0,6,15,18,85.0,96,565,184,...,3.6617,37,30500000,7.283,669,0,100,0,28850000.0,7.6
996,1944.0,0,97.0,6,19,0,78.0,588,391,865,...,1.6443,37,1000000,7.400,455,0,4,0,-1590000.0,7.6


In [304]:
# data.sample shuffles the data
# embaralha e divide em 80% treino e 20% teste

train, test = np.split(
    tabela_modelo.sample(frac=1, random_state=42),
    [int(0.8*len(tabela_modelo))]
)

X_train, y_train = scale_dataset(train, oversample=False)
X_test, y_test = scale_dataset(test, oversample=False)


/usr/local/lib/python3.12/dist-packages/numpy/_core/fromnumeric.py:57: FutureWarning:

'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.



In [305]:

# RandomForestRegressor

regressor = RandomForestRegressor(n_estimators=10, random_state=0, oob_score=True)

regressor.fit(X_train, y_train)

/usr/local/lib/python3.12/dist-packages/sklearn/ensemble/_forest.py:612: UserWarning:

Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable OOB estimates.



RandomForestRegressor(n_estimators=10, oob_score=True, random_state=0)

In [306]:
oob_score = regressor.oob_score_
print(f'Out-of-Bag Score: {oob_score}')

predictions = regressor.predict(X_test)

mse = mean_squared_error(y_test, predictions)
print(f'Root Squared Error: {mse}')

rms = root_mean_squared_error(y_test, predictions)
print(f'Root Mean Squared Error: {rms}')

rmsl = root_mean_squared_log_error(y_test, predictions)
print(f'Root Mean Squared Log Error: {rmsl}')

r2 = r2_score(y_test, predictions)
print(f'R-squared: {r2}')

Out-of-Bag Score: -11.481379395297225
Root Squared Error: 0.04458150000000007
Root Mean Squared Error: 0.21114331625699184
Root Mean Squared Log Error: 0.023022431479871347
R-squared: 0.40194181269388685


In [307]:
def plot_true_vs_pred(true_values, predicted_values, title='True vs Predicted'):
    """
    Plota um gráfico de dispersão usando Plotly comparando valores reais e previstos.

    Parâmetros:
    -----------
    true_values : array-like
        Valores reais.
    predicted_values : array-like
        Valores previstos pelo modelo.
    title : str
        Título do gráfico.
    """

    # Define limites para a linha y=x
    p1 = max(max(predicted_values), max(true_values))
    p2 = min(min(predicted_values), min(true_values))

    # Cria o gráfico de dispersão
    fig = go.Figure()

    # Pontos de dispersão
    fig.add_trace(go.Scatter(
        x=true_values,
        y=predicted_values,
        mode='markers',
        marker=dict(color='crimson'),
        name='Predicted vs True'
    ))

    # Linha de identidade
    fig.add_trace(go.Scatter(
        x=[p2, p1],
        y=[p2, p1],
        mode='lines',
        line=dict(color='blue', dash='dash'),
        name='y = x'
    ))

    # Layout
    fig.update_layout(
        title=title,
        xaxis=dict(title='True Values', type='log'),
        yaxis=dict(title='Predicted Values', type='log'),
        width=800,
        height=800,
        template='plotly_white',
        legend=dict(x=0.02, y=0.98)
    )

    fig.show()


In [308]:
plot_true_vs_pred(true_values=y_test,predicted_values=predictions)

In [309]:
# xg boost



X = tabela_modelo[tabela_modelo.columns[:-1]].values
y = tabela_modelo[tabela_modelo.columns[-1]].values
X_train, X_test_, y_train, y_test_ = train_test_split(X,y, test_size=0.25, random_state=0)
xgb_train = xgb.DMatrix(X_train, y_train, enable_categorical=True)
xgb_test = xgb.DMatrix(X_test_, y_test_, enable_categorical=True)
params = {
    'objective': 'reg:squarederror',
    'max_depth': 3,
    'learning_rate': 0.1,
}
n=50
model = xgb.train(params=params,dtrain=xgb_train,num_boost_round=n)
preds = model.predict(xgb_test)
#preds = np.round(preds)

mse = mean_squared_error(y_test_, preds)
print(f'Mean Squared Error: {mse}')

rms = root_mean_squared_error(y_test_, preds)
print(f'Root Mean Squared Error: {rms}')

rmsl = root_mean_squared_log_error(y_test_, preds)
print(f'Root Mean Squared Log Error: {rmsl}')

r2 = r2_score(y_test_, preds)
print(f'R-squared: {r2}')

Mean Squared Error: 0.016321643308454307
Root Mean Squared Error: 0.12775618696741972
Root Mean Squared Log Error: 0.014258203180060387
R-squared: 0.7639219477768329


In [310]:
plot_true_vs_pred(true_values=y_test_,predicted_values=preds)

# Prever o The Shawshank Redemption

In [315]:
shawshank_imdb_data = {'Series_Title': 'The Shawshank Redemption',
'Released_Year': '1994',
'Certificate': 'A',
'Runtime': '142 min',
'Genre': 'Drama',
'Meta_score': 80.0,
'Director': 'Frank Darabont',
'Star1': 'Tim Robbins',
'Star2': 'Morgan Freeman',
'Star3': 'Bob Gunton',
'Star4': 'William Sadler',
'No_of_Votes': 2343110,
'Gross': '28,341,469'}

shawshank_tmdb_data = {
  "adult": "false",
  "backdrop_path": "/pNjh59JSxChQktamG3LMp9ZoQzp.jpg",
  "belongs_to_collection": 0,
  "budget": 25000000,
  "genres": [
    {
      "id": 18,
      "name": "Drama"
    },
    {
      "id": 80,
      "name": "Crime"
    }
  ],
  "homepage": "",
  "id": 278,
  "imdb_id": "tt0111161",
  "origin_country": [
    "US"
  ],
  "original_language": "en",
  "original_title": "The Shawshank Redemption",
  "popularity": 27.6298,
  "poster_path": "/9cqNxx0GxF0bflZmeSMuL5tnGzr.jpg",
  "production_companies": [
    {
      "id": 97,
      "logo_path": "/7znWcbDd4PcJzJUlJxYqAlPPykp.png",
      "name": "Castle Rock Entertainment",
      "origin_country": "US"
    }
  ],
  "production_countries": [
    {
      "iso_3166_1": "US",
      "name": "United States of America"
    }
  ],
  "release_date": "1994-09-23",
  "revenue": 28341469,
  "runtime": 142,
  "spoken_languages": [
    {
      "english_name": "English",
      "iso_639_1": "en",
      "name": "English"
    }
  ],
  "status": "Released",
  "tagline": "Fear can hold you prisoner. Hope can set you free.",
  "title": "The Shawshank Redemption",
  "video": "false",
  "vote_average": 8.712,
  "vote_count": 28820
}

In [319]:
shawshank_input = {
    'Released_Year': int(shawshank_imdb_data['Released_Year']),
    'Certificate': shawshank_imdb_data.get('Certificate', ''),
    'Runtime (min)': int(shawshank_imdb_data['Runtime'].replace(' min','')),
    'Genre 1': shawshank_tmdb_data['genres'][0]['name'] if len(shawshank_tmdb_data['genres'])>0 else '',
    'Genre 2': shawshank_tmdb_data['genres'][1]['name'] if len(shawshank_tmdb_data['genres'])>1 else '',
    'Genre 3': shawshank_tmdb_data['genres'][2]['name'] if len(shawshank_tmdb_data['genres'])>2 else '',
    'Meta_score': float(shawshank_imdb_data.get('Meta_score',0)),
    'Star1': shawshank_imdb_data.get('Star1',''),
    'Star2': shawshank_imdb_data.get('Star2',''),
    'Star3': shawshank_imdb_data.get('Star3',''),
    'Star4': shawshank_imdb_data.get('Star4',''),
    'No_of_Votes': int(shawshank_imdb_data.get('No_of_Votes',0)),
    'Gross': int(shawshank_imdb_data.get('Gross','0').replace(',','')),
    'budget': int(shawshank_tmdb_data.get('budget',0)),
    'origin_country': shawshank_tmdb_data['origin_country'][0] if len(shawshank_tmdb_data['origin_country'])>0 else '',
    'original_language': shawshank_tmdb_data.get('original_language',''),
    'popularity': float(shawshank_tmdb_data.get('popularity',0)),
    'production_countries': shawshank_tmdb_data['production_countries'][0]['name'] if len(shawshank_tmdb_data['production_countries'])>0 else '',
    'revenue': int(shawshank_tmdb_data.get('revenue',0)),
    'vote_average': float(shawshank_tmdb_data.get('vote_average',0)),
    'vote_count': int(shawshank_tmdb_data.get('vote_count',0)),
    'belongs_to_collection_name': '',  # não pertence a coleção
    'production_companies_names': shawshank_tmdb_data['production_companies'][0]['name'] if len(shawshank_tmdb_data['production_companies'])>0 else '',
    'belongs_to_collection_bin': 0,  # mesmo caso acima
}

# Calculando profit overall igual ao tabela_geral
shawshank_input['profit overall'] = shawshank_input['Gross'] - shawshank_input['budget']

# Transformando em DataFrame
shawshank_df = pd.DataFrame([shawshank_input])




In [322]:

# --- Aplicar LabelEncoder existente ---
for col, le in label_encoders.items():
    shawshank_df[col] = shawshank_df[col].apply(lambda x: str(x))  # garante string
    shawshank_df[col] = shawshank_df[col].apply(lambda x: le.transform([x])[0] if x in le.classes_ else -1)


In [323]:
# Exemplo de previsão com XGBoost

xgb_input = xgb.DMatrix(shawshank_df)
pred_shawshank = model.predict(xgb_input)
print(f'Predicted IMDB Rating: {pred_shawshank[0]}')

Predicted IMDB Rating: 8.819954872131348


In [337]:
nota_real = 9.3
erro_shaw = nota_real - pred_shawshank
print("Erro para o The Shawshank Redemption",erro_shaw)

Erro para o The Shawshank Redemption [0.48004532]


In [333]:
len(tabela_geral[tabela_geral["IMDB_Rating"]>8.8])

7

In [336]:
print("Porcentagem de filmes com nota acima de 8.8 no imdb",100*len(tabela_geral[tabela_geral["IMDB_Rating"]>8.8])/len(tabela_geral),"%")

Porcentagem de filmes com nota acima de 8.8 no imdb 0.7014028056112225 %


# Salvar Modelo

In [338]:
# Caminho no seu Google Drive
model_path = "/content/drive/My Drive/Indicium/Trabalho Indicium/xgb_model.pkl"

# Salvar o modelo
with open(model_path, "wb") as f:
    pickle.dump(model, f)

# Salvar também os label encoders
encoders_path = "/content/drive/My Drive/Indicium/Trabalho Indicium/label_encoders.pkl"
with open(encoders_path, "wb") as f:
    pickle.dump(label_encoders, f)

In [ ]:
# Carregar o modelo
with open(model_path, "rb") as f:
    loaded_model = pickle.load(f)

# Carregar os encoders
with open(encoders_path, "rb") as f:
    loaded_label_encoders = pickle.load(f)


Boa, Vinicius 👌 já vi seu notebook com os trechos de modelagem no final — você treinou RandomForestRegressor e XGBoost, mostrou os prints de resultados e comparou, então o ponto 3 do desafio (modelagem) você cumpriu sim. Isso é muito bom.

Agora, para a entrega ficar 100% redonda e competitiva, falta basicamente organização, apresentação e alguns complementos pequenos:

🔧 O que ainda falta (checklist final)

Respostas explícitas às perguntas (Seção 2 do desafio)
Mesmo que seus gráficos e análises já estejam nos notebooks, é importante reservar seções dedicadas:

2a. Filme recomendado → usar critérios de nota + votos (ex.: “recomendo The Shawshank Redemption porque tem nota alta e muitos votos, ou outro critério seu”).

2b. Fatores para alto faturamento → correlacionar Gross com Votes, Runtime, Genre e destacar o que mais pesa.

2c. Insights do Overview → mostrar o que fez com NLP (TF-IDF, contagem de palavras-chave, clusters ou até só nuvem de palavras), e comentar se dá pra inferir gênero.

Predição para o caso do Shawshank Redemption (Seção 4)
Você já tem o modelo, falta só rodar a previsão para esse filme e colocar o resultado no notebook.

Salvar o modelo em .pkl (Seção 5)
Adicionar no final:

import joblib
joblib.dump(melhor_modelo, "modelo_imdb.pkl")


Isso mostra que você seguiu exatamente a entrega pedida.

README.md (Seção 6)
Explicar:

Objetivo do desafio.

Como instalar dependências:

pip install -r requirements.txt


Como rodar os notebooks.

Onde está o modelo .pkl.

requirements.txt
Gerar com:

pip freeze > requirements.txt


(ou montar só com as libs que usou: pandas, numpy, matplotlib, seaborn, scikit-learn, xgboost, joblib, requests, etc.).

Relatório final em PDF (ou exportar notebook)

Pode exportar o analise.ipynb já estruturado para PDF (File > Export As > PDF via LaTeX/nbconvert).

Ou montar um .md resumindo (eu posso estruturar pra você).

✨ Diferenciais que você já tem

Você foi além com dados externos da API TMDB (ótimo ponto extra).

Testou mais de um modelo e comparou resultados (muitos candidatos não fazem isso).

Documentou prints das métricas (mostra transparência).

👉 Ou seja: do ponto de vista técnico, você já entregou o essencial.
O que falta é packaging: responder claramente às perguntas, salvar o .pkl, gerar README + requirements + relatório. Isso é o que mais conta na entrega.

Quer que eu prepare pra você um modelo de README.md + esqueleto de relatório em markdown/PDF já adaptado para esse desafio, onde você só cola os resultados dos notebooks?

Exatamente, Vinicius ✅

O passo 4 do desafio pede explicitamente:

Supondo um filme com as seguintes características [...] qual seria a nota do IMDB?

Ou seja, você precisa rodar o seu modelo com os dados fornecidos no PDF (The Shawshank Redemption) e mostrar a predição.

In [309]:
# Salvar o modelo XGBoost treinado em formato .pkl
with open('xgb_model.pkl', 'wb') as f:
    pickle.dump(model, f)
